## First dataset and tokenization attempt

In [ ]:
with open("Harry_Potter_all_char_separated.txt", "r") as file:
    lines = []
    for line in file:
        lines = line.split("|")
    print(f"Number of lines :{len(lines)}")


Number of lines :79731


In [ ]:
with open("Harry_Potter_all_char_separated.txt", "r") as file:
    chars = []
    for line in file:
        for char in line:
            if char != "|":
                chars.append(char)

    print(f"Number of characters :{len(chars)}")



Number of characters :6435489


In [ ]:
from collections import Counter

counter = Counter(chars)
print(counter)

keys = counter.keys()
tokens = sorted(list(keys)) #character level tokenization
print(f"Nuber of tokens :{len(tokens)}\n")

for t in tokens:
    print(t, end="")

Counter({' ': 1338782, 'e': 570968, 't': 392008, 'a': 375735, 'o': 366717, 'n': 309034, 'r': 303917, 'i': 296392, 'h': 280449, 's': 278706, 'd': 231033, 'l': 198557, 'u': 138432, 'g': 117841, 'y': 112596, 'w': 107532, 'm': 103770, 'c': 96127, 'f': 90860, ',': 86100, 'p': 73341, 'b': 69118, '.': 62382, 'k': 53744, 'v': 39172, 'H': 38396, '“': 37038, '”': 36415, '’': 34770, 'I': 19208, 'T': 13178, 'S': 11660, 'M': 11096, '?': 10877, 'W': 10263, 'D': 10031, 'R': 9413, 'A': 9244, 'P': 7549, '!': 6943, 'x': 6365, 'B': 6176, 'G': 6098, 'C': 5357, 'N': 4946, 'j': 4860, 'L': 4855, 'Y': 4788, 'F': 4704, 'z': 4468, 'O': 4453, 'E': 4194, 'q': 3495, 'V': 2394, 'U': 2069, 'K': 1824, ':': 1630, 'Q': 822, 'J': 804, '‘': 468, ')': 379, '(': 327, '1': 178, 'Z': 127, 'X': 127, '2': 37, '3': 29, '4': 23, '7': 21, '9': 20, '0': 17, '5': 15, '6': 13, '8': 11, '&': 1})
Nuber of tokens :75

 !&(),.0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz‘’“”

## New Dataset of Wine Reviews and tokenization method

In [1]:
!pip install kagglehub

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zynicide/wine-reviews")

print("Path to dataset files:", path)

100%|██████████| 50.9M/50.9M [00:01<00:00, 39.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4


In [3]:
import os
files = [file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))]
print(files)

['winemag-data-130k-v2.csv', 'winemag-data-130k-v2.json', 'winemag-data_first150k.csv']


In [4]:
import pandas as pd

data = pd.read_csv(f"{path}/winemag-data-130k-v2.csv")

# all categories
#print(data.columns.tolist())

print("Number of rows:", data.shape[0])
print("Missing values:\n", data.isnull().sum())


Number of rows: 129971
Missing values:
 Unnamed: 0                   0
country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64


In [ ]:
#remove row with missing values
cleaned_data = data.dropna()
print("Number of rows after cleaning:", cleaned_data.shape[0])


#too less rows after dropping the rows with no data, so decided to fill the missing values insted

Number of rows after cleaning: 22387


In [5]:
#Filling missing data
data.fillna("", inplace=True)
print("Number of rows:", data.shape[0])

missing_values = data.isnull().sum()
print(missing_values)

<ipython-input-5-f33e01b52957>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("", inplace=True)


Number of rows: 129971
Unnamed: 0               0
country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
dtype: int64


In [6]:
data["processed_text"] = (
    "wine review: " + data["country"] + " : " + data["province"] + " : " + data["variety"] + " : " + data["description"]
)


print(data["processed_text"].head())

0    wine review: Italy : Sicily & Sardinia : White...
1    wine review: Portugal : Douro : Portuguese Red...
2    wine review: US : Oregon : Pinot Gris : Tart a...
3    wine review: US : Michigan : Riesling : Pineap...
4    wine review: US : Oregon : Pinot Noir : Much l...
Name: processed_text, dtype: object


In [7]:
!pip install transformers


### Bert - word pair tokenizer

⚠️ Not suitable for text generation, even I dont know why I spent time on this

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the processed text
data["tokens"] = data["processed_text"].apply(lambda x: tokenizer(x, padding=True, truncation=True, max_length=512))

# Example: View the first tokenized row
print("First Tokenized Entry:")
print(data["tokens"].iloc[0])

First Tokenized Entry:
{'input_ids': [101, 4511, 3319, 1024, 3304, 1024, 12071, 1004, 21594, 1024, 2317, 12586, 1024, 23958, 2015, 2421, 5133, 5909, 1010, 23528, 1010, 7987, 5714, 9221, 1998, 9550, 12810, 1012, 1996, 14412, 3686, 3475, 1005, 1056, 15241, 22570, 1010, 5378, 4895, 29443, 6675, 6207, 1010, 20418, 1998, 9550, 10878, 4077, 28022, 5648, 3012, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
index = 3
processed_text = data["processed_text"].iloc[index]
input_ids = data["tokens"].iloc[index]["input_ids"]
decoded_tokens = [tokenizer.decode([id]) for id in input_ids]
decoded_text = tokenizer.decode(data["tokens"].iloc[index]["input_ids"])


# Print the results
print("Processed Text:")
print(processed_text)
print("\nInput IDs:")
print(input_ids)
print("\nDecoded Tokens:")
print(decoded_tokens)
print("\nDecoded Text:")
print(decoded_text)

Processed Text:
wine review: US : Michigan : Riesling : Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.

Input IDs:
[101, 4511, 3319, 1024, 2149, 1024, 4174, 1024, 15544, 2229, 2989, 1024, 7222, 23804, 15544, 4859, 1010, 14380, 6770, 2232, 1998, 4589, 20593, 2707, 2125, 1996, 23958, 2015, 1012, 1996, 14412, 3686, 2003, 1037, 2978, 2062, 6728, 27581, 1010, 2007, 3964, 1997, 6861, 1011, 2852, 10993, 29247, 2094, 19739, 12462, 1998, 24792, 3228, 2126, 2000, 1037, 3621, 2004, 18886, 25997, 2102, 1010, 4100, 21190, 3926, 1012, 102]

Decoded Tokens:
['[CLS]', 'wine', 'review', ':', 'us', ':', 'michigan', ':', 'ri', '##es', '##ling', ':', 'pine', '##apple', 'ri', '##nd', ',', 'lemon', 'pit', '##h', 'and', 'orange', 'blossom', 'start', 'off', 'the', 'aroma', '##s', '.', 'the', 'pal', '##ate', 'is', 'a', 'bit', 'more', 'op', '##ulent', ',', 'wit

In [ ]:
from collections import Counter

# Flatten all input_ids from your dataset
all_input_ids = [id for tokens in data["tokens"] for id in tokens["input_ids"]]

# Count token occurrences
token_counts = Counter(all_input_ids)

# Map token IDs back to text
id_to_token = {id: tokenizer.decode([id]) for id in token_counts.keys()}

# Create a list of tuples (Token Text, Token ID, Count) and sort by frequency
sorted_tokens_by_frequency = sorted(
    [(id_to_token[token_id], token_id, count) for token_id, count in token_counts.items()],
    key=lambda x: x[2],  # Sort by count (frequency)
    reverse=True         # Descending order
)

print(f"Total Tokens: {total_tokens}")
print(f"Unique Tokens: {len(sorted_tokens_by_frequency)}")
print("\nTop 20 Most Frequent Tokens:")
for token_text, token_id, count in sorted_tokens_by_frequency[:20]:
    print(f"Token: {token_text}, Token ID: {token_id}, Count: {count}")


Total Tokens: 9236768
Unique Tokens: 16620

Top 20 Most Frequent Tokens:
Token: :, Token ID: 1024, Count: 521322
Token: ,, Token ID: 1010, Count: 441931
Token: ., Token ID: 1012, Count: 356241
Token: and, Token ID: 1998, Count: 348172
Token: the, Token ID: 1996, Count: 221530
Token: wine, Token ID: 4511, Count: 212149
Token: a, Token ID: 1037, Count: 180002
Token: of, Token ID: 1997, Count: 173073
Token: review, Token ID: 3319, Count: 129981
Token: [CLS], Token ID: 101, Count: 129971
Token: [SEP], Token ID: 102, Count: 129971
Token: with, Token ID: 2007, Count: 120532
Token: this, Token ID: 2023, Count: 114197
Token: ##s, Token ID: 2015, Count: 97905
Token: is, Token ID: 2003, Count: 96866
Token: -, Token ID: 1011, Count: 89615
Token: it, Token ID: 2009, Count: 86121
Token: ##y, Token ID: 2100, Count: 70794
Token: in, Token ID: 1999, Count: 63212
Token: flavors, Token ID: 26389, Count: 62812


### GPT2 - BPE tokenizer

In [8]:
%%time

from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token #to fix error wit padding


# tested max_length of this dataset, should be fine as now row is longer than 512 hence there wont be any truncation
# learnt padding is important for GPU - but need help here
data["tokens"] = data["processed_text"].apply(
    lambda x: tokenizer(x, padding=True, truncation=True, max_length=512, return_tensors="pt")
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

CPU times: user 1min 28s, sys: 1.55 s, total: 1min 29s
Wall time: 1min 49s


In [9]:
index = 0

processed_text = data["processed_text"].iloc[index]
input_ids = data["tokens"].iloc[index]["input_ids"].squeeze().tolist()  # Convert to a flat list
decoded_tokens = [tokenizer.decode([id]) for id in input_ids]
decoded_text = tokenizer.decode(input_ids)

print("Processed Text:")
print(processed_text)
print("\nInput IDs:")
print(input_ids)
print("\nDecoded Tokens:")
print(decoded_tokens)
print("\nDecoded Text:")
print(decoded_text)


Processed Text:
wine review: Italy : Sicily & Sardinia : White Blend : Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.

Input IDs:
[39002, 2423, 25, 8031, 1058, 49301, 1222, 46997, 43168, 1058, 2635, 41198, 1058, 317, 398, 292, 2291, 19690, 8234, 11, 47085, 11, 49943, 6440, 290, 16577, 21047, 13, 383, 44100, 2125, 470, 17698, 38084, 11, 6011, 555, 5528, 2945, 17180, 11, 35405, 290, 16577, 35021, 7848, 35984, 7408, 414, 13]

Decoded Tokens:
['wine', ' review', ':', ' Italy', ' :', ' Sicily', ' &', ' Sard', 'inia', ' :', ' White', ' Blend', ' :', ' A', 'rom', 'as', ' include', ' tropical', ' fruit', ',', ' broom', ',', ' brim', 'stone', ' and', ' dried', ' herb', '.', ' The', ' palate', ' isn', "'t", ' overly', ' expressive', ',', ' offering', ' un', 'rip', 'ened', ' apple', ',', ' citrus', ' and', ' dried', ' sage', ' alongside', ' brisk', ' acid', 'ity', '.']

De

In [ ]:
#checking if there are rows more than 512 tokens - Result: No rows exceed 512 tokens. (atleast for this dataset)
# don't run this
rows_with_long_tokens = []

for index, row in data.iterrows():
    tokenized = tokenizer(row["processed_text"], truncation=False)
    if len(tokenized["input_ids"]) > 512:
        rows_with_long_tokens.append((index, row["processed_text"], len(tokenized["input_ids"])))

# Display rows with more than 512 tokens
if rows_with_long_tokens:
    long_token_data = pd.DataFrame(rows_with_long_tokens, columns=["Index", "Processed Text", "Token Count"])
else:
    print("No rows exceed 512 tokens.")

No rows exceed 512 tokens.


In [12]:
from collections import Counter

# Flatten all input_ids from the tokenized data
all_input_ids = [
    token_id
    for tokens in data["tokens"]
    for token_id in tokens["input_ids"].squeeze().tolist()
]

# Count token occurrences
token_counts = Counter(all_input_ids)

# Map token IDs back to text
id_to_token = {token_id: tokenizer.decode([token_id]) for token_id in token_counts.keys()}

# Create a list of tuples (Token Text, Token ID, Count) and sort by frequency
sorted_tokens_by_frequency = sorted(
    [(id_to_token[token_id], token_id, count) for token_id, count in token_counts.items()],
    key=lambda x: x[2],  # Sort by count (frequency)
    reverse=True         # Descending order
)

# Convert to DataFrame for visualization
max_rows = 50
top_tokens = pd.DataFrame(sorted_tokens_by_frequency[:max_rows], columns=["Token Text", "Token ID", "Count"])

# Display summary
total_tokens = sum(token_counts.values())
print(f"Total Tokens: {total_tokens}")
print(f"Unique Tokens: {len(sorted_tokens_by_frequency)}")
print(f"\nTop {max_rows} Most Frequent Tokens:")
print(top_tokens)


Total Tokens: 9044974
Unique Tokens: 23051

Top 50 Most Frequent Tokens:
     Token Text  Token ID   Count
0             ,        11  440537
1             :      1058  389915
2             .        13  355484
3           and       290  347182
4            of       286  172870
5           the       262  168174
6             a       257  157732
7             :        25  131406
8          wine     39002  130064
9        review      2423  129979
10         with       351  115651
11           is       318   96695
12            -        12   89417
13         wine      8237   80093
14         this       428   72883
15           to       284   61426
16           in       287   60612
17      flavors     17361   60316
18           US      1294   54506
19          The       383   52648
20           's       338   51481
21        fruit      8234   48682
22          ity       414   44576
23            t       256   44541
24           It       632   43595
25           on       319   43095
26       

In [13]:
print(data["tokens"].iloc[0])

{'input_ids': tensor([[39002,  2423,    25,  8031,  1058, 49301,  1222, 46997, 43168,  1058,
          2635, 41198,  1058,   317,   398,   292,  2291, 19690,  8234,    11,
         47085,    11, 49943,  6440,   290, 16577, 21047,    13,   383, 44100,
          2125,   470, 17698, 38084,    11,  6011,   555,  5528,  2945, 17180,
            11, 35405,   290, 16577, 35021,  7848, 35984,  7408,   414,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
